In [1]:
import pypropep as ppp
ppp.init()

Loaded 1921 thermo species
Loaded 1030 propellants


In [2]:
print sorted(ppp.PROPELLANTS.keys())[0:5]

['1,1,1-TRINITRO-2-HYDROXYBUTYRI C ACID', '1,1,1-TRINTRO-4,4-BIS(DIFLUORO AMION)PENTANE', '1,1,1TRINITRO-2-HYDROXYBUTANOL', '1,1-DIMETHYL HYDRAZINE NITRATE', '1,2,4-BUTANETRIOL TRINITRATE']


In [3]:
e = ppp.Equillibrium()
o2 = ppp.PROPELLANTS['OXYGEN (GAS)']
ch4 = ppp.PROPELLANTS['METHANE']
e.add_propellants([(o2, 1.), (ch4, 1.)])
print e

cpropep composition:
	OXYGEN (GAS) - 1.000 mol
	METHANE - 1.000 mol

